# Deepmind - i3d

In [1]:
import tensorflow_hub as tfhub
import tensorflow as tf
import numpy as np
import cv2 as cv
import re

In [2]:
import ssl
from urllib import request

In [3]:
url_dataset = 'https://www.crcv.ucf.edu/data/UCF101/UCF101.rar'
url_labelmap = 'https://raw.githubusercontent.com/deepmind/kinetics-i3d/master/data/label_map.txt'

In [4]:
ssl_cert = ssl._create_unverified_context()

In [ ]:
# Alternativo
names = request.urlopen(url_dataset, context=ssl_cert).read().decode('utf-8')
videos = re.findall('(v_[\w]+\.avi)', names)
videos = set(videos)

In [5]:
with request.urlopen(url_labelmap) as file:
  labels = [line.decode('utf-8').strip() for line in file]

In [14]:
def read_video(path, max_frames=32, shape=(224,224)):
  video = cv.VideoCapture(path)
  frames = []
  while len(frames)<max_frames:
    flag, frame = video.read()
    if not flag: break
    frame = cv.resize(frame, shape)
    frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    frames.append(frame)
  video.release()
  frames = np.array(frames) / 255.
  return frames

In [22]:
def predict(model, labels, video):
  video_input = tf.constant(video, dtype=tf.float32)
  video_input = video_input[tf.newaxis, ...]
  logits = model(video_input)['default'][0]
  probs = tf.nn.softmax(logits)
  for i in np.argsort(probs)[::-1][:3]:
    print(labels[i]+':',probs[i])
  return probs

In [12]:
model_url = 'https://tfhub.dev/deepmind/i3d-kinetics-400/1'
model = tfhub.load(model_url)

In [17]:
model = model.signatures['default']

## Dataset

In [8]:
! mkdir ~/.kaggle
! cp drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [9]:
! kaggle datasets download toxtli/video-blooper-dataset-for-automatic-video-editing

100% 2.25G/2.25G [00:19<00:00, 143MB/s]
100% 2.25G/2.25G [00:19<00:00, 125MB/s]


In [ ]:
! unzip /content/video-blooper-dataset-for-automatic-video-editing.zip
! rm /content/video-blooper-dataset-for-automatic-video-editing.zip

prediccion

In [26]:
video = read_video('/content/bloopers/Test/utterance_65.mp4', max_frames=32, shape=(224,224))

In [27]:
probs = predict(model, labels, video)

stretching arm: tf.Tensor(0.2779601, shape=(), dtype=float32)
tying bow tie: tf.Tensor(0.06360261, shape=(), dtype=float32)
tying tie: tf.Tensor(0.0568181, shape=(), dtype=float32)
